In [1]:
# Importing libraries
import pandas as pd
from pathlib import Path
import requests
import json
import folium
from folium.plugins import HeatMap


# Import the API key
from config import geoapify_key


In [2]:
# Importing data
censorship_data_csv = Path("Resources/Censorship_Data_by_Country.csv")
population_data_csv = Path("Resources/Country_Population_2023.csv")

# Reading data
censorship_data = pd.read_csv(censorship_data_csv)
population_data = pd.read_csv(population_data_csv)

In [3]:
# Inspecting data for Censorship
censorship_data.head()

,Flag,Country,Score,Torrents Restricted?,Torrents Banned or Shut Down?,Pornography Restricted?,Pornography Banned?,Political Media Restricted?,Political Media Heavily Censored?,Social Media Restricted?,Social Media Banned?,VPNs Restricted,VPNs Banned?,Messaging and VoIP App Restrictions
0,:af:,Afghanistan,6,Y,N,Y,Y,Y,Y,Y,N,N,N,N
1,:al:,Albania,2,Y,N,N,N,Y,N,N,N,N,N,N
2,:dz:,Algeria,5,Y,N,Y,N,Y,Y,Y,N,N,N,N
3,:ad:,Andorra,1,Y,N,N,N,N,N,N,N,N,N,N
4,:ao:,Angola,2,Y,N,N,N,Y,N,N,N,N,N,N


In [4]:
# Inspecting data for Population
population_data.head()

,#,Country (or dependency),Population (2023)
0,1,India,"1,428,627,663"
1,2,China,"1,425,671,352"
2,3,United States,"339,996,563"
3,4,Indonesia,"277,534,122"
4,5,Pakistan,"240,485,658"


In [5]:
# Cleaning and Renaming Population Data
population_data = population_data.rename(columns={"Country (or dependency)":"Country"})
population_data = population_data.rename(columns={"Population (2023)":"Population"})
population_data = population_data.drop(columns=["#"])
population_data.head()


,Country,Population
0,India,"1,428,627,663"
1,China,"1,425,671,352"
2,United States,"339,996,563"
3,Indonesia,"277,534,122"
4,Pakistan,"240,485,658"


In [10]:
# Merging Censorship and Population
censor_data_df = pd.merge(censorship_data, population_data, on="Country", how="left")
censor_data_df.head()

censor_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Flag                                 174 non-null    object
 1   Country                              175 non-null    object
 2   Score                                175 non-null    int64 
 3   Torrents Restricted?                 175 non-null    object
 4   Torrents Banned or Shut Down?        175 non-null    object
 5   Pornography Restricted?              175 non-null    object
 6   Pornography Banned?                  175 non-null    object
 7   Political Media Restricted?          175 non-null    object
 8   Political Media Heavily Censored?    175 non-null    object
 9   Social Media Restricted?             175 non-null    object
 10  Social Media Banned?                 175 non-null    object
 11  VPNs Restricted                      175 non-

In [17]:
# Removed null values in the dataset
df_clean = censor_data_df.dropna()
df_clean.info()
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, 0 to 174
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Flag                                 165 non-null    object
 1   Country                              165 non-null    object
 2   Score                                165 non-null    int64 
 3   Torrents Restricted?                 165 non-null    object
 4   Torrents Banned or Shut Down?        165 non-null    object
 5   Pornography Restricted?              165 non-null    object
 6   Pornography Banned?                  165 non-null    object
 7   Political Media Restricted?          165 non-null    object
 8   Political Media Heavily Censored?    165 non-null    object
 9   Social Media Restricted?             165 non-null    object
 10  Social Media Banned?                 165 non-null    object
 11  VPNs Restricted                      165 non-null 

,Flag,Country,Score,Torrents Restricted?,Torrents Banned or Shut Down?,Pornography Restricted?,Pornography Banned?,Political Media Restricted?,Political Media Heavily Censored?,Social Media Restricted?,Social Media Banned?,VPNs Restricted,VPNs Banned?,Messaging and VoIP App Restrictions,Population
0,:af:,Afghanistan,6,Y,N,Y,Y,Y,Y,Y,N,N,N,N,"42,239,854"
1,:al:,Albania,2,Y,N,N,N,Y,N,N,N,N,N,N,"2,832,439"
2,:dz:,Algeria,5,Y,N,Y,N,Y,Y,Y,N,N,N,N,"45,606,480"
3,:ad:,Andorra,1,Y,N,N,N,N,N,N,N,N,N,N,"80,088"
4,:ao:,Angola,2,Y,N,N,N,Y,N,N,N,N,N,N,"36,684,202"


In [15]:
# Extract Country into a list
country_list = list(df_clean["Country"])
print(country_list)


['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Djibouti', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', '

In [44]:
# Geoapify Base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [72]:
# Prepare a list to hold the results
results = []

# Loop through the country list and get coordinates
for country in country_list:
    
    # Set up the parameters for the request
    params = {
        'country': country,
        'apiKey': geoapify_key,
        'limit': 1  # We only want the first (most relevant) result
    }
    
    # Make the request to the Geoapify API
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:

        # Parse the response JSON to get the coordinates
        data = response.json()
        if data['features']:

            # Usually, the first feature in the list is the most relevant
            feature = data['features'][0]
            coordinates = feature['geometry']['coordinates']
            region = feature['properties']['timezone']

            # Add the country and its coordinates to the results list
            results.append({
                'Country': country,
                'Latitude': coordinates[1],
                'Longitude': coordinates[0],
                'Region' : region['name']
            })
        else:
            print(f'No results found for {country}')
    else:
        print(f'Failed to get coordinates for {country}: HTTP {response.status_code}')

# Create a DataFrame from the results
df_coordinates = pd.DataFrame(results)

# Display the DataFrame
print(df_coordinates)


         Country   Latitude   Longitude            Region
0    Afghanistan  33.768006   66.238514        Asia/Kabul
1        Albania  41.000028   19.999962     Europe/Tirane
2        Algeria  28.000027    2.999983    Africa/Algiers
3        Andorra  42.540717    1.573203    Europe/Andorra
4         Angola -11.877577   17.569124     Africa/Luanda
..           ...        ...         ...               ...
160    Venezuela   8.001871  -66.110932   America/Caracas
161      Vietnam  15.926666  107.965086  Asia/Ho_Chi_Minh
162        Yemen  16.347124   47.891527         Asia/Aden
163       Zambia -14.518912   27.558988     Africa/Lusaka
164     Zimbabwe -18.455496   29.746841     Africa/Harare

[165 rows x 4 columns]


In [74]:
# Splitting the 'Region' column at '/' and keeping only the first part
df_coordinates['Region'] = df_coordinates['Region'].str.split('/').str[0]
df_coordinates.head()

,Country,Latitude,Longitude,Region
0,Afghanistan,33.768006,66.238514,Asia
1,Albania,41.000028,19.999962,Europe
2,Algeria,28.000027,2.999983,Africa
3,Andorra,42.540717,1.573203,Europe
4,Angola,-11.877577,17.569124,Africa


In [76]:
# Merging Censor Data, with Coordinates Data
final_df = pd.merge(df_clean, df_coordinates, on="Country", how="inner")
final_df.head(50)

,Flag,Country,Score,Torrents Restricted?,Torrents Banned or Shut Down?,Pornography Restricted?,Pornography Banned?,Political Media Restricted?,Political Media Heavily Censored?,Social Media Restricted?,Social Media Banned?,VPNs Restricted,VPNs Banned?,Messaging and VoIP App Restrictions,Population,Latitude,Longitude,Region
0,:af:,Afghanistan,6,Y,N,Y,Y,Y,Y,Y,N,N,N,N,"42,239,854",33.768006,66.238514,Asia
1,:al:,Albania,2,Y,N,N,N,Y,N,N,N,N,N,N,"2,832,439",41.000028,19.999962,Europe
2,:dz:,Algeria,5,Y,N,Y,N,Y,Y,Y,N,N,N,N,"45,606,480",28.000027,2.999983,Africa
3,:ad:,Andorra,1,Y,N,N,N,N,N,N,N,N,N,N,"80,088",42.540717,1.573203,Europe
4,:ao:,Angola,2,Y,N,N,N,Y,N,N,N,N,N,N,"36,684,202",-11.877577,17.569124,Africa
5,:ar:,Argentina,3,Y,Y,N,N,Y,N,N,N,N,N,N,"45,773,884",-34.996496,-64.967282,America
6,:am:,Armenia,3,Y,N,Y,Y,N,N,N,N,N,N,N,"2,777,970",40.769627,44.673665,Asia
7,:au:,Australia,3,Y,Y,Y,N,N,N,N,N,N,N,N,"26,439,111",-24.776109,134.755000,Australia
8,:at:,Austria,2,Y,Y,N,N,N,N,N,N,N,N,N,"8,958,960",47.522617,14.143702,Europe
9,:az:,Azerbaijan,5,Y,N,Y,N,Y,Y,Y,N,N,N,N,"10,412,651",40.393629,47.787251,Asia


In [78]:

# Initialize a map:
map = folium.Map(location=[final_df['Latitude'].mean(), final_df['Longitude'].mean()], zoom_start=2)

# Create heat_data list with Latitude, Longitude and Score as weight
heat_data = [[row['Latitude'], row['Longitude'], row['Score']] for index, row in final_df.iterrows()]

# Add heat map layer to the map:
heatmap = HeatMap(heat_data, max_val=max(final_df['Score']), min_opacity=0.25, radius=50, blur=15, 
                  max_zoom=1, use_local_extrema=True)
map.add_child(heatmap)

# Iterate over each row of the DataFrame and add a CircleMarker
for idx, row in final_df.iterrows():
    folium.CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=5, # Small radius for the marker
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=folium.Popup(f"Country: {row['Country']}<br>Region: {row['Region']}<br>Score: {row['Score']}<br>Population: {row['Population']}", max_width=300)
    ).add_to(map)



display(map)

C:\Users\Developer\AppData\Local\Temp\ipykernel_29272\857156641.py:8: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  heatmap = HeatMap(heat_data, max_val=max(final_df['Score']), min_opacity=0.25, radius=50, blur=15,
